<a href="https://colab.research.google.com/github/lin3372/400_gan/blob/main/books/generative_deep_learning/0303_vae_digits_train_(iL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup

fix for [TypeError: Could not build a TypeSpec for <KerasTensor: shape=(None, None, 4)](https://github.com/matterport/Mask_RCNN/issues/2458) that origially shown in VAE.train(.....)

In [1]:
!pip uninstall keras-preprocessing
!pip install git+https://github.com/keras-team/keras-preprocessing.git
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.compat.v1.get_default_graph()

Found existing installation: Keras-Preprocessing 1.1.2
Uninstalling Keras-Preprocessing-1.1.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/Keras_Preprocessing-1.1.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/keras_preprocessing/*
Proceed (y/n)? y
  Successfully uninstalled Keras-Preprocessing-1.1.2
  Cloning https://github.com/keras-team/keras-preprocessing.git to /tmp/pip-req-build-1ze9jb_5
  Running command git clone -q https://github.com/keras-team/keras-preprocessing.git /tmp/pip-req-build-1ze9jb_5
  Created wheel for Keras-Preprocessing: filename=Keras_Preprocessing-1.1.2-py3-none-any.whl size=43632 sha256=21751490bd124a0a5ec73b04cc1a3fe7cd87c05f5f06930623d27fa2ba282774
  Stored in directory: /tmp/pip-ephem-wheel-cache-h78253o7/wheels/6d/8c/1d/f7c16ffd97b199cf4a20b04f1b9444a9390272edafb52c45f7
Successfully built Keras-Preprocessing
Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
GOOGLE_COLAB=0
USE_GPU=0

## gpu

In [3]:
try:
  !nvidia-smi
except:
  print("couldn not communicate with the NVIDIA driver")

%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  #raise SystemError('GPU device not found')
  print('GPU device not found, using CPU')
else:
  USE_GPU=1
  print('Found GPU at: {}'.format(device_name))


Sat Jan 15 09:41:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## mount google drive



In [4]:
import os, sys
if 'google.colab' in sys.modules:
  print("Running in Google Colab, mounting GDrive")
  from google.colab import drive
  drive.mount('/gdrive')
  GOOGLE_COLAB=1

  PROJ_DATA_DIR = "/gdrive/MyDrive/github/400_gan/books/generative_deep_learning/"
  os.makedirs(PROJ_DATA_DIR, exist_ok=True)


Running in Google Colab, mounting GDrive
Mounted at /gdrive


## working directory setting

1. With Colab this prog will clone from davidADSP/GDL_code.git and revised in lin3372/400_gan 
2. With local machine this prog will be running in the directory and the cloned repo will be in "./src/"

 directory
* The **data** folder is where to download relevant data sources (chapter 3 onwards)
* The **run** folder stores output from the generative models (chapter 3 onwards) 
* The **utils** folder stores useful functions that are sourced by the main notebooks

In [5]:
if (GOOGLE_COLAB==1): ## when executing from PC
  PROJ_WORKING_DIR = "/gdrive/MyDrive/github/400_gan/books/generative_deep_learning/src/"
else:  ## when executing from PC, assuming running from where the inpynb is located
  PROJ_WORKING_DIR = "./src/"

os.makedirs(PROJ_WORKING_DIR, exist_ok=True)  

## Clone/copy from Github/GDrive

In [6]:
import os
import shutil 
import sys 

!git config --global user.email "lin3372@gmail.com"
!git config --global user.name "lin3372"

if os.path.exists("./400_gan/") and os.path.isdir("./400_gan/"):
  shutil.rmtree("./400_gan/")
if os.path.exists("./src/") and os.path.isdir("./src/"):
  shutil.rmtree("./src/")

if (GOOGLE_COLAB==1):
  try:
    !git clone https://github.com/lin3372/400_gan 
    #!git clone https://github.com/davidADSP/GDL_code.git
  except:
    print("github.com/lin3372/400_gan  clone failed!")      
  else:
    print("github.com/lin3372/400_gan  cloned!")

  PROJ_GIT_SRC_PATH="./400_gan/books/generative_deep_learning/src/"
else: # running on PC
  PROJ_DAT_PATH="./src/"

shutil.copytree(PROJ_GIT_SRC_PATH, "./src")

os.chdir("./src/")

Cloning into '400_gan'...
remote: Enumerating objects: 452, done.
remote: Counting objects: 100% (452/452), done.
remote: Compressing objects: 100% (208/208), done.
remote: Total 452 (delta 308), reused 328 (delta 236), pack-reused 0
Receiving objects: 100% (452/452), 16.82 MiB | 10.31 MiB/s, done.
Resolving deltas: 100% (308/308), done.
github.com/lin3372/400_gan  cloned!


# VAE Training

## imports

In [7]:
import os

from models.VAE import VariationalAutoencoder
from utils.loaders import load_mnist

In [8]:
# run params
SECTION = 'vae'
RUN_ID = '0002'
DATA_NAME = 'digits'
RUN_FOLDER = PROJ_WORKING_DIR + 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #

## data

In [9]:
(x_train, y_train), (x_test, y_test) = load_mnist()
print(f"x_train={x_train.shape}")
print(f"y_train={y_train.shape}")
print(f"x_test={x_test.shape}")
print(f"y_test={y_test.shape}")

11501568/11490434 [==============================] - 0s 0us/step
x_train=(60000, 28, 28, 1)
y_train=(60000,)
x_test=(10000, 28, 28, 1)
y_test=(10000,)


## architecture

In [10]:
vae = VariationalAutoencoder(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64, 64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_t_filters = [64,64,32,1]
    , decoder_conv_t_kernel_size = [3,3,3,3]
    , decoder_conv_t_strides = [1,2,2,1]
    , z_dim = 2
)

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [11]:
vae.encoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 encoder_conv_0 (Conv2D)        (None, 28, 28, 32)   320         ['encoder_input[0][0]']          
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 28, 28, 32)   0           ['encoder_conv_0[0][0]']         
                                                                                                  
 encoder_conv_1 (Conv2D)        (None, 14, 14, 64)   18496       ['leaky_re_lu[0][0]']            
                                                                                            

In [12]:
vae.decoder.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 dense (Dense)               (None, 3136)              9408      
                                                                 
 reshape (Reshape)           (None, 7, 7, 64)          0         
                                                                 
 decoder_conv_t_0 (Conv2DTra  (None, 7, 7, 64)         36928     
 nspose)                                                         
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 7, 7, 64)          0         
                                                                 
 decoder_conv_t_1 (Conv2DTra  (None, 14, 14, 64)       36928     
 nspose)                                                   

## training

In [13]:
LEARNING_RATE = 0.0005
R_LOSS_FACTOR = 1000

In [14]:
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [15]:
BATCH_SIZE = 32
EPOCHS = 200
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [16]:
import datetime
T0 = datetime.datetime.now()
print("Starting @ " + str(T0))

Starting @ 2022-01-15 09:42:30.155869


In [17]:
vae.train(     
    x_train
    , batch_size = BATCH_SIZE
    , epochs = EPOCHS
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)

Train on 60000 samples
Epoch 1/200
  704/60000 [..............................] - ETA: 15:03 - loss: 213.1812 - vae_r_loss: 212.8701 - vae_kl_loss: 0.3110

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


59776/60000 [============================>.] - ETA: 0s - loss: 58.5448 - vae_r_loss: 55.3064 - vae_kl_loss: 3.2384
Epoch 00001: saving model to /gdrive/MyDrive/github/400_gan/books/generative_deep_learning/src/run/vae/0002_digits/weights/weights-001-58.52.h5

Epoch 00001: saving model to /gdrive/MyDrive/github/400_gan/books/generative_deep_learning/src/run/vae/0002_digits/weights/weights.h5
60000/60000 [==============================] - 25s 421us/sample - loss: 58.5198 - vae_r_loss: 55.2795 - vae_kl_loss: 3.2404 - lr: 5.0000e-04
Epoch 2/200
59744/60000 [============================>.] - ETA: 0s - loss: 51.7444 - vae_r_loss: 47.8212 - vae_kl_loss: 3.9232
Epoch 00002: saving model to /gdrive/MyDrive/github/400_gan/books/generative_deep_learning/src/run/vae/0002_digits/weights/weights-002-51.74.h5

Epoch 00002: saving model to /gdrive/MyDrive/github/400_gan/books/generative_deep_learning/src/run/vae/0002_digits/weights/weights.h5
60000/60000 [==============================] - 16s 271us/sa

# save/copy the results

In [18]:
#GDRIVE_LOG_FOLDER = PROJ_WORKING_DIR + RUN_FOLDER
#os.makedirs(GDRIVE_LOG_FOLDER, exist_ok=True)  
#os.system(f"cp -R {RUN_FOLDER}/* {GDRIVE_LOG_FOLDER}")
#print(f"cp -R {RUN_FOLDER}/* {GDRIVE_LOG_FOLDER}")

In [19]:
T1 = datetime.datetime.now()
print("Starting  @ " + str(T0))
print("Finishing @ " + str(T1))

Starting  @ 2022-01-15 09:42:30.155869
Finishing @ 2022-01-15 10:41:52.511531
